In [1]:
import os

In [2]:
from keras.preprocessing import image

In [3]:
import cv2

In [4]:
categories = ['with_mask','without_mask']

In [5]:
data = []
for category in categories:
    path = os.path.join('train',category)

    label = categories.index(category)
    
    for file in os.listdir(path):
        
        img_path = os.path.join(path,file)
        img = cv2.imread(img_path)
        img = cv2.resize(img,(224,224))

        data.append([img,label])

        

In [6]:
len(data)

1376

In [7]:
import random

In [8]:
random.shuffle(data)

In [9]:
X = []
y = []
for features,label in data:
    X.append(features)
    y.append(label)

In [10]:
len(X)

1376

In [11]:
len(y)

1376

In [12]:
import numpy as np

In [13]:
X = np.array(X) 
y = np.array(y)

In [14]:
X.shape

(1376, 224, 224, 3)

In [15]:
y.shape

(1376,)

In [16]:
y

array([1, 1, 0, ..., 1, 0, 0])

In [17]:
X = X/255

In [18]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2)

In [19]:
X_train.shape

(1100, 224, 224, 3)

In [20]:
X_test.shape

(276, 224, 224, 3)

In [21]:
from keras.applications.vgg16 import VGG16

In [22]:
vgg = VGG16()

In [23]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [24]:
from keras import Sequential

In [25]:
model = Sequential()

In [26]:
for layer in vgg.layers[:-1]:
    model.add(layer)

In [27]:
for layer in model.layers:
    layer.trainable=False

In [28]:
from keras.layers import Dense

In [29]:
model.add(Dense(1,activation='sigmoid'))

In [30]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)       2

In [31]:
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])

In [33]:
model.fit(X_train,y_train,epochs=5, validation_data=(X_test,y_test))

Epoch 1/5
35/35 [==============================] - 212s 6s/step - loss: 0.5797 - accuracy: 0.7136 - val_loss: 0.3969 - val_accuracy: 0.9130
Epoch 2/5
35/35 [==============================] - 192s 6s/step - loss: 0.3382 - accuracy: 0.9200 - val_loss: 0.2930 - val_accuracy: 0.9167
Epoch 3/5
35/35 [==============================] - 198s 6s/step - loss: 0.2656 - accuracy: 0.9227 - val_loss: 0.2633 - val_accuracy: 0.9022
Epoch 4/5
35/35 [==============================] - 196s 6s/step - loss: 0.2195 - accuracy: 0.9309 - val_loss: 0.1922 - val_accuracy: 0.9384
Epoch 5/5
35/35 [==============================] - 227s 7s/step - loss: 0.2015 - accuracy: 0.9427 - val_loss: 0.1709 - val_accuracy: 0.9565


In [34]:
cap = cv2.VideoCapture(0)

In [43]:
while  True:
    ret, frame = cap.read()

    img = cv2.resize(frame,(224,224))

    y_pred = detect_face_mask(img)

    coods = detect_face(cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY))

    for x,y,w,h in coods:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),3)
        
    if y_pred == 0:
        draw_label(frame,"mask",(30,30),(0,255,0))

    else:
        draw_label(frame,"No mask",(30,30),(0,0,255))


    cv2.imshow("window",frame)
    if cv2.waitKey(1) & 0xFF == ord("x"):
        break
cv2.destroyAllWindows()

1/1 [==============================] - 0s 197ms/step


In [52]:
from sklearn.metrics import confusion_matrix
import itertools
from keras.applications import xception

In [56]:
xception_bf = xception.Xception(weights='imagenet', include_top=False, pooling='avg')
bf_train_x = xception_bf.predict(X_train, batch_size=32, verbose=2)
bf_train_val = xception_bf.predict(X_test, batch_size=32, verbose=2)

35/35 - 98s - 98s/epoch - 3s/step
9/9 - 22s - 22s/epoch - 2s/step


In [57]:
predictions = np.argmax(model.predict(bf_train_val),axis=1)

ValueError: in user code:

    File "C:\Users\tansu\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\tansu\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\tansu\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\tansu\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 2079, in predict_step
        return self(x, training=False)
    File "C:\Users\tansu\AppData\Roaming\Python\Python39\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\tansu\AppData\Roaming\Python\Python39\site-packages\keras\e

In [60]:
confusion_mat = confusion_matrix(y_test, y_train)

plt.figure(figsize=(4,4))
sns.heatmap(confusion_mat, square=True, annot=True,
            yticklabels=['wear mask', 'without mask'],
            xticklabels=['wear mask', 'without mask']);
plt.title('CONFUSION MATRIX');
plt.xlabel('Y_TRUE');
plt.ylabel("PREDICTIONS");


ValueError: Found input variables with inconsistent numbers of samples: [276, 1100]

In [36]:
def detect_face_mask(img):
    y_pred = (model.predict(img.reshape(1,224,224,3))> 0.5).astype("int32")
    return y_pred[0][0]

In [37]:
sample1 = cv2.imread('sample1.jpg')
sample1 = cv2.resize(sample1,(224,224))

In [38]:
detect_face_mask(sample1)

1/1 [==============================] - 3s 3s/step


0

In [39]:
def draw_label(img,text,pos,bg_color):
    text_size = cv2.getTextSize(text,cv2.FONT_HERSHEY_SIMPLEX,1,cv2.FILLED)
    end_x = pos[0] + text_size[0][0] + 2
    end_y = pos[1] + text_size[0][1] + 2

    cv2.rectangle(img,pos,(end_x,end_y),bg_color,cv2.FILLED)
    cv2.putText(img,text,pos,cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,0),1,cv2.LINE_AA)

In [40]:
haar = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [42]:
def detect_face(img):
    coods = haar.detectMultiScale(img)
    return coods